In [1]:
import psycopg2
import pandas as pd

In [2]:
connection = psycopg2.connect(
    host        = 'localhost',
    port        = '5433',
    user        = 'postgres',
    password    = 'postgres',
    database    = 'TuDu'
)

connection

<connection object at 0x000001F923589030; dsn: 'user=postgres password=xxx dbname=TuDu host=localhost port=5433', closed: 0>

In [3]:
cursor = connection.cursor()

In [ ]:
# def get_connection() / init_db

In [31]:
def view_tasks(user_id):   
    query = f'''SELECT task_name FROM tasks WHERE user_id = {user_id}'''
    tasks = pd.read_sql(query, connection)
    return tasks

view_tasks(4)

C:\Users\Admin\AppData\Local\Temp\ipykernel_19384\3622710056.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tasks = pd.read_sql(query, connection)


,task_name
0,Einkaufen
1,Praxisprojekt SQL
2,Trinken
3,Steuererklärung
4,Kita Termin vereinbaren


In [23]:
def view_tasks_clean(user_id):
    query = f'SELECT * FROM tasks'
    cursor.execute(query, (user_id,))
    tasks = [task[0] for task in cursor.fetchall()]
    return tasks

view_tasks_clean(4)


[1, 2, 3, 4, 5]

In [4]:
def view_tasks(user_id):   
    query = f'SELECT task_name FROM tasks WHERE user_id = %s'
    cursor.execute(query, (user_id,))
    columns = [desc[0] for desc in cursor.description]
    tasks = cursor.fetchall()
    return pd.DataFrame(tasks, columns=columns)


print(view_tasks(4))    

                 task_name
0                Einkaufen
1        Praxisprojekt SQL
2                  Trinken
3          Steuererklärung
4  Kita Termin vereinbaren


In [34]:
def view_tasks(user_id):
    query = 'SELECT task_id, task_name, completed FROM tasks WHERE user_id = %s'
    cursor.execute(query, (user_id,))
    tasks = cursor.fetchall()
    return tasks 

tasks = view_tasks(4)
tasks


[(1, 'Einkaufen', False),
 (2, 'Praxisprojekt SQL', False),
 (3, 'Trinken', False),
 (4, 'Steuererklärung', False),
 (5, 'Kita Termin vereinbaren', False)]

In [35]:
def update_task_status(task_name):
    cursor.execute('UPDATE tasks SET completed = True WHERE task_id = %s',(task_name,))
    connection.commit()